# Time Series Profiler

In [2]:
import os
import tsgm
import keras
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

2025-06-20 06:21:19.018314: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-20 06:21:19.079533: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-20 06:21:19.418858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-20 06:21:19.418990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-20 06:21:19.487169: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## Select a scenario
Set the SCENARIOS index value from the available scenarios

In [3]:
data = tsgm.utils.get_energy_data()
data = data[0]

n, m = data.shape
window = 24*6  # hourly
pop = n % window
data = data[pop:]
n, m = data.shape
windowed = np.array(np.split(data, n/window))

scaler = tsgm.utils.TSFeatureWiseScaler()
scaled_data = scaler.fit_transform(windowed)

In [ ]:
model = tsgm.models.timeGAN.TimeGAN(
    seq_len=window,
    module="gru",
    hidden_dim=window,
    n_features=m,
    n_layers=3,
    batch_size=256,
    gamma=1.0,
)
# .compile() sets all optimizers to Adam by default
model.compile()

model.fit(
    data=scaled_data,
    epochs=100,
)

model.save_weights(path)

Model: "Autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RealData (InputLayer)       [(256, 144, 28)]          0         
                                                                 
 Embedder (Sequential)       (256, 144, 144)           346608    
                                                                 
 Recovery (Sequential)       (256, 144, 28)            379900    
                                                                 
Total params: 726508 (2.77 MB)
Trainable params: 726508 (2.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "AdversarialSupervised"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RandomNoise (InputLayer)    [(256, 144, 28)]          0         
                                                      

INFO:models:Start Embedding Network Training
Autoencoder - training: 100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:37<00:00,  2.65it/s]
INFO:models:Finished Embedding Network Training
INFO:models:Start Training with Supervised Loss Only
Adversarial Supervised - training: 100%|████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.63it/s]
INFO:models:Finished Training with Supervised Loss Only
INFO:models:Start Joint Training
GAN with embedding - training:  38%|████████████████████████▋                                        | 38/100 [10:36<18:10, 17.59s/it]

In [ ]:
new_data = model.generate(n_samples=len(windowed))

In [ ]:
print(new_data[0][:,0])
synth = scaler.inverse_transform(new_data[0])

In [ ]:
cols = ["Appliances", "lights", "T1", "RH_1", "T2", "RH_2",
        "T3", "RH_3","T4", "RH_4", "T5", "RH_5", "T6",
        "RH_6", "T7", "RH_7", "T8", "RH_8", "T9", "RH_9",
        "T_out", "Press_mm_hg", "RH_out", "Windspeed",
        "Visibility", "Tdewpoint", "rv1", "rv2"
]

# Plotting some generated samples. Both Synthetic and Original data are still standardized with values between [0, 1]
fig, axes = plt.subplots(nrows=7, ncols=4, figsize=(15, 10))
axes=axes.flatten()

time = range(window)
for j, col in enumerate(cols):
    df = pd.DataFrame({'Real': windowed[0][:, j],
                   'Synthetic': synth[:, j]})
    df.plot(ax=axes[j],
            title = col,
            secondary_y='Synthetic data', style=['-', '--'])
fig.tight_layout()

if not os.path.exists("./img"):
    os.makedirs("./img")
plt.savefig('img/comparison_gan_outputs.png', dpi=400)

In [ ]:
tsgm.utils.visualize_tsne_unlabeled(scaled_data, new_data, perplexity=10, markersize=100, alpha=0.5)